<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Maratona_BehindTheCode_IBM20/blob/main/Desafio%206/DF6_Lit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install tpot

     |████████████████████████████████| 92kB 3.7MB/s 
     |████████████████████████████████| 163kB 13.9MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=400da203109eb0c6b4146857947a78c038205d7a822625210a357c9df9a89e7f
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [ ]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score

from tpot import TPOTClassifier

In [ ]:
df = pd.read_csv("https://github.com/maratonadev-br/desafio-6-2020/blob/master/dataset/training_dataset.csv?raw=true")
test = pd.read_csv("https://raw.githubusercontent.com/maratonadev-br/desafio-6-2020/master/dataset/to_be_scored.csv")

df.shape, test.shape

((15857, 15), (1000, 14))

## PreProcessing

In [ ]:
colsToDrop = ["id", "importante_ter_certificado"
              #"profissao", "graduacao", "modulos_iniciados", 
              #"pretende_fazer_cursos_lit", "como_conheceu_lit",
              #"universidade", "organizacao"
              ]
for col in colsToDrop:
  try: 
    df.drop(col, axis=1, inplace=True)
    test.drop(col, axis=1, inplace=True)
  except: print(f"{col} already droped")

In [ ]:
df_num = df[["certificados", "modulos_finalizados", "modulos_iniciados", "total_modulos",
             "categoria"]]
df_num = df_num.dropna()
df_num.shape

(9618, 5)

In [ ]:
df_dropedna = df.dropna()

In [ ]:
colsNumber = df.select_dtypes(include="number").columns
df[colsNumber] = df[colsNumber].fillna(0)

In [ ]:
df["graduacao"].fillna("SEM FORMAÇÃO", inplace=True)
df["profissao"].fillna("SEM EXPERIÊNCIA", inplace=True)
df["como_conheceu_lit"].fillna("OUTROS", inplace=True)
df["organizacao"].fillna("Eletroeletronicos", inplace=True)
df["universidade"].fillna("FATEC", inplace=True)

In [ ]:
colsToDummy = ['universidade', "organizacao", "como_conheceu_lit",
               'graduacao', 'profissao'] 
# df = pd.get_dummies(df, columns=colsToDummy)

In [ ]:
le = LabelEncoder()
df_dropedna[colsToDummy] = df_dropedna[colsToDummy].apply(lambda x: le.fit_transform(x)).astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_dropedna

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
0,1,13,2,1,1.0,1.0,6.0,1,28.0,5.0,4.0,0.0,perfil6
2,0,12,8,9,1.0,1.0,10.0,7,27.0,27.0,27.0,0.0,perfil2
9,0,16,7,7,0.0,0.0,9.0,6,51.0,49.0,49.0,0.0,perfil2
10,5,17,1,1,1.0,1.0,3.0,1,10.0,5.0,1.0,0.0,perfil6
11,2,10,0,7,0.0,0.0,2.0,0,10.0,3.0,1.0,0.0,perfil6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15839,4,11,8,3,0.0,0.0,4.0,3,19.0,15.0,14.0,0.0,perfil4
15843,5,7,6,2,1.0,1.0,8.0,6,133.0,102.0,101.0,3.0,perfil1
15844,0,19,9,7,0.0,0.0,6.0,6,97.0,39.0,32.0,0.0,perfil2
15845,2,6,4,4,1.0,1.0,2.0,3,273.0,16.0,10.0,0.0,perfil4


## Tentando Balancear

In [ ]:
df["categoria"].value_counts()

perfil6    4539
perfil2    4215
perfil1    2472
perfil5    1917
perfil4    1661
perfil3    1053
Name: categoria, dtype: int64

# Training

In [ ]:
X = df[["certificados", "modulos_finalizados", "modulos_iniciados", "total_modulos"]]
# X = df_dropedna.drop("categoria", axis=1)
# X = df_num.drop("categoria", axis=1)
# y = df_num["categoria"]
y = df["categoria"]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=.7, random_state=0)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((11099, 4), (4758, 4), (11099,), (4758,))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

Xtrain_smote, ytrain_smote = smote.fit_resample(Xtrain, ytrain)
Xtrain_smote.shape, ytrain_smote.shape

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

((19218, 4), (19218,))

In [ ]:
tp_smote = TPOTClassifier(scoring="f1_micro", random_state=0, verbosity=2, config_dict="TPOT light")
tp_smote.fit(Xtrain_smote, ytrain_smote)
tp_smote("pipeline_smote")


Generation 1 - Current best internal CV score: 0.820377956542471

TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=10, min_samples_leaf=17, min_samples_split=3)


TypeError: ignored

In [ ]:
tp = TPOTClassifier(scoring="f1_micro", random_state=0, verbosity=2)
tp.fit(Xtrain, ytrain)
tp.export("pipeline3")


Generation 1 - Current best internal CV score: 0.8351171664289474
Generation 2 - Current best internal CV score: 0.8351172767395709

TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.01, max_depth=10, max_features=0.25, min_samples_leaf=15, min_samples_split=10, n_estimators=100, subsample=0.8)


In [ ]:
from xgboost import XGBClassifier

# f1_micro: 0.8317582241150573 | ((2431, 12), (1043, 12), (2431,), (1043,))
exported_pipeline = XGBClassifier(learning_rate=0.1, max_depth=1, 
                                  min_child_weight=17, n_estimators=100, 
                                  subsample=0.7, random_state=0)

exported_pipeline.fit(Xtrain, ytrain)
results = exported_pipeline.predict(Xtest)
print(f1_score(ytest, results, average="micro"))
# 0.8475398475398476 oi? | ((6732, 4), (2886, 4), (6732,), (2886,))

0.8475398475398476


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# f1_micro: 0.8351172767395709 | ((6732, 4), (2886, 4), (6732,), (2886,))
exported_pipeline = GradientBoostingClassifier(learning_rate=0.01, max_depth=10, 
                                               max_features=0.25, min_samples_leaf=15, 
                                               min_samples_split=10, n_estimators=100, 
                                               subsample=0.8, random_state=0)
exported_pipeline.fit(Xtrain, ytrain)
results = exported_pipeline.predict(Xtest)
print(f1_score(ytest, results, average="micro"))

0.8482328482328483


In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from tpot.export_utils import set_param_recursive

# f1_micro: 0.8351171664289474 | ((6732, 4), (2886, 4), (6732,), (2886,))
exported_pipeline = make_pipeline(
    SelectPercentile(score_func=f_classif, percentile=61),
    DecisionTreeClassifier(criterion="entropy", max_depth=4, min_samples_leaf=3, min_samples_split=14)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 0)

exported_pipeline.fit(Xtrain, ytrain)
results = exported_pipeline.predict(Xtest)
print(f1_score(ytest, results, average="micro"))

0.8496188496188496


In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import gp_minimize

def tunar_modelo(params):
  learning_rate = params[0]
  max_depth = params[1]
  min_child_weight = params[2]
  n_estimators = params[3]
  subsample = params[4]

  print(params,'\n')
  mdl = XGBClassifier(  
      learning_rate = learning_rate, n_estimators = n_estimators,
      max_depth = max_depth, min_child_weight = min_child_weight,
      subsample = subsample, random_state = 0)
  
  mdl.fit(Xtrain, ytrain)
  p = mdl.predict(Xtest)
  return -f1_score(ytest, p, average="micro")

space = [(1e-2, 1e-1), # learning_rate
         (1, 1000), # n_estimators
         (1, 100), # max_depth
         (1, 100), # min_child_weight, 
         (0, 1)] # subsample

resultado_gp = gp_minimize(tunar_modelo, space, random_state=0, 
                           n_calls=50, n_random_starts=20, verbose=1)

In [ ]:
resultado_gp.x # 0.8496 [0.1, 1000, 70, 100, 1] 

[0.06335601564025166, 844, 86, 85, 1]

In [ ]:
xgb = XGBClassifier( 
      learning_rate = 0.1, n_estimators = 1000,
      max_depth = 70, min_child_weight = 100,
      subsample = 1, random_state = 0)
xgb.fit(Xtrain, ytrain)
xgb_p = xgb.predict(Xtest)
print(f1_score(ytest, xgb_p, average="micro"))

0.8496188496188496


In [ ]:
xgb2 = XGBClassifier( 
      learning_rate = 0.01, n_estimators = 100,
      max_depth = 8, min_child_weight = 20,
      subsample = 0.45, random_state = 0)
xgb2.fit(Xtrain, ytrain)
xgb2_p = xgb.predict(Xtest)
print(f1_score(ytest, xgb2_p, average="micro"))

0.8496188496188496


In [ ]:
xgb2_smote = XGBClassifier( 
      learning_rate = 0.01, n_estimators = 100,
      max_depth = 8, min_child_weight = 20,
      subsample = 0.45, random_state = 0)
xgb2_smote.fit(Xtrain, ytrain)
xgb2_smote_p = xgb.predict(Xtest)
print(f1_score(ytest, xgb2_smote_p, average="micro"))

0.7635561160151324


## Predict test

In [ ]:
# test[colsToDummy] = test[colsToDummy].apply(lambda x: le.fit_transform(x)).astype(int)
test2 = test[["certificados", "modulos_finalizados", "modulos_iniciados", "total_modulos"]]

In [ ]:
results2 = xgb2.predict(test2)

In [ ]:
results1 = pd.read_csv("/content/results.csv")

In [ ]:
results1.shape[0], results2.shape[0]

(1000, 1000)

In [ ]:
count = 0
for row in range(0, 1000):
  if results1["target"][row] != results2[row]:
    print(f"{results1['target'][row]} != {results2[row]}, {row}")
    count += 1
print(count)

perfil4 != perfil6, 408
1


---

In [ ]:
results_smote = exported_pipeline.predict(test2)

In [ ]:
count = 0
for row in range(0, 1000):
  if results1["target"][row] != results_smote[row]:
    print(f"{results1['target'][row]} != {results_smote[row]}, {row}")
    count += 1
print(count)

perfil1 != perfil6, 336
perfil4 != perfil6, 408
2


In [ ]:
test_results_smote = pd.DataFrame({"target":results_smote})
test_results_smote

,target
0,perfil6
1,perfil3
2,perfil6
3,perfil1
4,perfil2
...,...
995,perfil1
996,perfil2
997,perfil6
998,perfil2


In [ ]:
test_results_smote.to_csv("results", index=False)